In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
sentenceData = spark.createDataFrame([(0,'Hi I heared about Spark'),
                                (1, 'I wish Java could use case classes'),
                                (2, 'Logistic, regression, models, are, neat')],
                               ['id','sentence'])

In [5]:
sentenceData.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heared about...|
|  1|I wish Java could...|
|  2|Logistic, regress...|
+---+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [7]:
words_data = tokenizer.transform(sentenceData)

In [8]:
words_data.show(truncate=False)

+---+---------------------------------------+---------------------------------------------+
|id |sentence                               |words                                        |
+---+---------------------------------------+---------------------------------------------+
|0  |Hi I heared about Spark                |[hi, i, heared, about, spark]                |
|1  |I wish Java could use case classes     |[i, wish, java, could, use, case, classes]   |
|2  |Logistic, regression, models, are, neat|[logistic,, regression,, models,, are,, neat]|
+---+---------------------------------------+---------------------------------------------+



In [9]:
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heared about...|[hi, i, heared, a...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic, regress...|[logistic,, regre...|
+---+--------------------+--------------------+



In [10]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [11]:
featurized_data = hashing_tf.transform(words_data)

In [12]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [13]:
idf_model = idf.fit(featurized_data)

In [14]:
rescaled_data = idf_model.transform(featurized_data)

In [20]:
rescaled_data.select('features').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                      |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(262144,[18700,19036,33808,112576,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                  |
|(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|(262144,[91006,148320,158779,20099

In [15]:
from pyspark.ml.feature import CountVectorizer

In [16]:
df = spark.createDataFrame([(0,"a b c".split(" ")),
                           (1, "a b b c a".split(" "))],
                          ["id","words"])

In [17]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [18]:
cv = CountVectorizer(inputCol="words", outputCol="features",
                    vocabSize=3, minDF=2.0)

In [19]:
model = cv.fit(df)

In [20]:
result = model.transform(df)

In [21]:
# Bag of words
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

